In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# TODO: ADD BUCKET NAME!
bucket_name = 'YOUR_BUCKET_NAME_HERE'

In [ ]:
# If some files are in the way you can rerun this.
full_path = f"gs://{bucket_name}/"
paths=[]
blobs = client.list_blobs(bucket_name)
client = storage.Client()
for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("postings_gcp"):
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

In [ ]:
# Count number of wiki pages
# I will want to use this later for tf-idf
parquetFile.count()

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS


In [ ]:
def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      Tuple
        (doc_id, len_after_removing_of_words)
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    # Filter out stopwords
    tokens = [token for token in tokens if token not in all_stopwords]
    # Calculate the length after removing stopwords
    len_after_removing_of_words = len(tokens)
    return (id, len_after_removing_of_words)

In [ ]:

# Calculate word counts for all documents
word_counts = doc_text_pairs.map(lambda x: word_count(x.text, x.id))

# Convert RDD to DataFrame
word_counts_df = word_counts.toDF(['id', 'word_count'])

# Write word counts to CSV in Google Cloud Storage
output_path = f'gs://{bucket_name}/word_counts'
word_counts_df.repartition(1).write.csv(output_path, compression="gzip")
